In [15]:
import pandas as pd

In [18]:
my_prediction = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\predictions_vs_real_with_variant_all_genes_updated_221123.csv")
# change column name from "reality" to "pathogenicity"
my_prediction.rename(columns={"reality": "pathogenicity"}, inplace=True)
my_prediction

,gene,variant,prediction,pathogenicity,protein_type
0,COL11A1,G1249V,1,1,collagen
1,COL11A1,G1516V,1,1,collagen
2,COL11A1,R1733C,1,1,collagen
3,COL11A1,G1800C,1,1,collagen
4,COL11A1,R1076C,1,1,collagen
...,...,...,...,...,...
1091,COL9A2,G30R,1,0,collagen
1092,COL9A2,A336E,1,0,collagen
1093,COL9A2,L335V,0,0,collagen
1094,COL9A2,R32Q,0,0,collagen


In [17]:
for gene in my_prediction["gene"].unique():
    print(f"Gene: {gene}")
    print(my_prediction[my_prediction["gene"] == gene]["pathogenicity"].value_counts())

Gene: COL11A1
pathogenicity
1    33
0     9
Name: count, dtype: int64
Gene: COL4A3
pathogenicity
1    127
0     18
Name: count, dtype: int64
Gene: COL9A3
pathogenicity
0    15
1     5
Name: count, dtype: int64
Gene: COL11A2
pathogenicity
1    19
0    13
Name: count, dtype: int64
Gene: COL4A5
pathogenicity
1    407
0     13
Name: count, dtype: int64
Gene: COL2A1
pathogenicity
1    286
0     10
Name: count, dtype: int64
Gene: COL4A4
pathogenicity
1    99
0    21
Name: count, dtype: int64
Gene: COL9A1
pathogenicity
0    5
1    1
Name: count, dtype: int64
Gene: COL9A2
pathogenicity
0    14
Name: count, dtype: int64
Gene: COL4A6
pathogenicity
0    1
Name: count, dtype: int64


In [4]:
tools = ['VEST', 'alphamissense', 'REVEL', 'EVE', 'MutPred']

In [5]:
# Create merge, which is the my_prediction dataframe with the predictions of the other tools
merged = my_prediction
for tool in tools:
    tool_predictions = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\{tool}_on_dvd_data_predictions_LOPO.csv")
    # Merge the two files
    merged = pd.merge(merged, tool_predictions, on=["gene", "variant"])
    # remove duplicate rows
    merged = merged.drop_duplicates(subset=["gene", "variant"])
# change column name of pathogenicity_x to pathogenicity
merged = merged.rename(columns={"prediction_x": "prediction", "am_pathogenicity": "am_score", "am_class": "alphamissense_pathogenicity"})

MergeError: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x'} is not allowed.

In [6]:
# Print how many variants there are from each gene

merged["gene"].value_counts()
# my_prediction["gene"].value_counts()
# print how many variants there are from each gene and each pathogenicity
for gene in merged["gene"].unique():
    print(f"Gene: {gene}")
    print(merged[merged["gene"] == gene]["pathogenicity"].value_counts())

Gene: COL2A1
pathogenicity
0    310
1    262
Name: count, dtype: int64
Gene: COL4A3
pathogenicity
0    146
1     95
Name: count, dtype: int64
Gene: COL4A5
pathogenicity
1    363
0    111
Name: count, dtype: int64
Gene: MYO7A
pathogenicity
0    473
1    161
Name: count, dtype: int64
Gene: FGFR1
pathogenicity
0    128
1    101
Name: count, dtype: int64


In [7]:
# Print all rows in merged that aren't in my_prediction, according to the variant
merged[merged["variant"].isin(my_prediction["variant"])]

,variant,prediction_x,pathogenicity,gene,Unnamed: 0_x,VEST_score,VEST_pathogenicity,Unnamed: 0_y,prediction_y,reality,uniprot_id,am_pathogenicity,am_class,Unnamed: 0,revel_score,REVEL_pathogenicity
0,M1R,0,0,COL2A1,0,0.905,1,0,0,0,P02458,0.1676,0,0,0.699,1
1,I2L,0,0,COL2A1,2,0.244,0,2,0,0,P02458,0.0908,0,1,0.247,0
2,I2T,0,0,COL2A1,4,0.116,0,4,0,0,P02458,0.2038,0,2,0.245,0
3,I2V,0,0,COL2A1,5,0.068,0,5,0,0,P02458,0.1286,0,3,0.184,0
4,L4F,0,0,COL2A1,8,0.073,0,8,0,0,P02458,0.0530,0,4,0.223,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,T695I,1,1,FGFR1,3495,0.984,1,4235,1,1,P11362,0.9884,1,3020,0.927,1
2444,A520T,1,1,FGFR1,3496,0.908,1,4236,1,1,P11362,0.8397,1,3021,0.843,1
2445,L712P,1,1,FGFR1,3497,0.975,1,4237,1,1,P11362,0.9997,1,3022,0.959,1
2446,E692G,1,1,FGFR1,3498,0.938,1,4238,1,1,P11362,0.9981,1,3023,0.986,1


In [8]:

### Notice that VEST4 doesn't have data of WFS1, and REVEL doesn't have data for SLC26A4.

In [9]:
## Calculate the MCC for the predictions, according to the real pathogenicity (0 or 1)
from sklearn.metrics import matthews_corrcoef

In [10]:
# separate the merged dataframe according to gene
genes = merged["gene"].unique()
print(f"Number of genes: {len(genes)}")

Number of genes: 5


In [11]:
# Calculate MCC for each gene specific predictor for mutpred

# Build dictionary with gene names as keys.
mccs = {gene: 0 for gene in genes}

for tool in tools:
    for gene in genes:
        gene_df = merged[merged["gene"] == gene]
        # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
        # Create binary predictions based on the 0.5 threshold
        
        gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
        
        # Get TP, TN, FP, FN
        TP = len(gene_df[(gene_df['pathogenicity'] == 1) & (gene_df[f"{tool}_pathogenicity"] == 1)])
        TN = len(gene_df[(gene_df['pathogenicity'] == 0) & (gene_df[f"{tool}_pathogenicity"] == 0)])
        FP = len(gene_df[(gene_df['pathogenicity'] == 0) & (gene_df[f"{tool}_pathogenicity"] == 1)])
        FN = len(gene_df[(gene_df['pathogenicity'] == 1) & (gene_df[f"{tool}_pathogenicity"] == 0)])
        print(f"{tool} --{gene} TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
        
        # Calculate MCC
        mcc = matthews_corrcoef(gene_df['pathogenicity'], gene_df[f"{tool}_pathogenicity"])
        
        # Get gene name for the use for the dictionary
        gene = gene_df['gene'].unique()[0]
        
        # Append mcc to dictionary
        mccs[gene] = mcc
    
    print(f"MCCs of {tool} predictions for each gene:")
    for gene in mccs:
        print(f"{gene}: {mccs[gene]}")
        
# get MCC for my predictions
mccs = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
    # Create binary predictions based on the 0.5 threshold
    
    gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
    gene_df['prediction'] = gene_df['prediction'].astype(int)
    
    # Get TP, TN, FP, FN
    TP = len(gene_df[(gene_df['pathogenicity'] == 1) & (gene_df['prediction'] == 1)])
    TN = len(gene_df[(gene_df['pathogenicity'] == 0) & (gene_df['prediction'] == 0)])
    FP = len(gene_df[(gene_df['pathogenicity'] == 0) & (gene_df['prediction'] == 1)])
    FN = len(gene_df[(gene_df['pathogenicity'] == 1) & (gene_df['prediction'] == 0)])
    
    print(f"my tool: {gene} TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
    
    # Calculate MCC
    mcc = matthews_corrcoef(gene_df['pathogenicity'], gene_df['prediction'])
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    
    # Append mcc to dictionary
    mccs[gene] = mcc

print(f"MCCs of predictions for each gene:")
for gene in mccs:
    print(f"{gene}: {mccs[gene]}")


C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_6372\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_6372\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_6372\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

VEST --COL2A1 TP: 261, TN: 297, FP: 13, FN: 1
VEST --COL4A3 TP: 94, TN: 138, FP: 8, FN: 1
VEST --COL4A5 TP: 363, TN: 111, FP: 0, FN: 0
VEST --MYO7A TP: 160, TN: 408, FP: 65, FN: 1
VEST --FGFR1 TP: 100, TN: 116, FP: 12, FN: 1
MCCs of VEST predictions for each gene:
COL2A1: 0.951720033257657
COL4A3: 0.9244710289855537
COL4A5: 1.0
MYO7A: 0.7790244298650963
FGFR1: 0.8903092234354874


KeyError: 'alphamissense_pathogenicity'

In [14]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, log_loss, confusion_matrix
# store the results in a dictionary
results = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    y_true = gene_df["pathogenicity"].astype(int)
    predicted_probabilities = gene_df["MutPred_score"]
    gene_df.loc[:, 'binary_prediction_mutpred'] = (gene_df['MutPred_score'] > 0.5).astype(int)
    predicted_labels = gene_df["binary_prediction_mutpred"]
    
    results_gene = {}
    # Calculate AUC-ROC
    roc_auc = roc_auc_score(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["roc_auc"] = roc_auc
    
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
    roc_auc = auc(fpr, tpr)
    # append to dictionary
    results_gene["auc"] = roc_auc
    
    # Calculate AUC-PR
    precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
    pr_auc = auc(recall, precision)
    # append to dictionary
    results_gene["pr_auc"] = pr_auc
    
    # Calculate F1 Score
    f1 = f1_score(y_true, predicted_labels)
    # append to dictionary
    results_gene["f1"] = f1
    
    # Calculate Log Loss
    logloss = log_loss(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["logloss"] = logloss
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_true, predicted_labels)
    # append to dictionary
    results_gene["confusion_matrix"] = conf_matrix
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    # Append results to dictionary
    results[gene] = results_gene
    
## Print results
print("Results of MutPred2 predictions for each gene:")
for gene in results:
    print(f"{gene}: {results[gene]}")

KeyError: 'MutPred_score'

In [12]:
# present the confusion matrix for each gene
import matplotlib.pyplot as plt

for gene in results:
    # Plot confusion matrix
    plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.xticks([0, 1], ["Benign", "Pathogenic"])
    plt.yticks([0, 1], ["Benign", "Pathogenic"])
    plt.title(f"Confusion matrix for {gene}, MutPred2 predictions")
    plt.style.use("seaborn-white")
    # Add text annotations
    for i in range(2):
        for j in range(2):
            plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black", backgroundcolor="white")
    plt.colorbar()
    plt.show()

NameError: name 'results' is not defined

In [13]:
# present the ROC curve for each gene
for gene in results:
    # Plot ROC curve
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC curve for {gene}, MutPred2 predictions")
    plt.legend(loc="lower right")
    plt.show()

NameError: name 'results' is not defined